# Developing and Evaluating Chained Classifier

In [32]:
from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.calibration import CalibratedClassifierCV as CCCV
from sklearn.calibration import calibration_curve
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix as confusion
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score as recall
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.multioutput import ClassifierChain
from sklearn.utils import resample
from xgboost import XGBClassifier as xgb
import itertools
import ipywidgets as widgets
import joblib
import numpy as np
import pandas as pd
import pip
import shap
import warnings
InteractiveShell.ast_node_interactivity = "all"

In [2]:
Xi=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\Xi.csv')
Xi_hold=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\Xi_hold.csv')
y=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y.csv')
y_hold=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold.csv')
y_hold_nssi=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_nssi.csv')
y_hold_si=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_si.csv')

In [3]:
#Dropping the sitbs column as we are only interested in predicting each component outcome with the chained classifier
y=y.drop(columns='sitbs')
y_hold=y_hold.drop(columns='sitbs')

In [5]:
RF=RandomForestClassifier(n_estimators=250, min_samples_split=50, max_features=200, random_state=26)
baseclf=CCCV(RF, method='isotonic')
chain=ClassifierChain(baseclf, order=None, random_state=26)

In [6]:
chain.fit(Xi, y)

ClassifierChain(base_estimator=CalibratedClassifierCV(estimator=RandomForestClassifier(max_features=200,
                                                                                       min_samples_split=50,
                                                                                       n_estimators=250,
                                                                                       random_state=26),
                                                      method='isotonic'),
                random_state=26)

In [32]:
#Creating function that predicts probabilities using the calibrated model
def predict(model, data):
    proba=model.predict_proba(data)
    print('Probs: %.3f (%.3f)' % (mean(proba), std(proba)) )

    return proba

In [9]:
testprob=predict(chain, Xi_hold)

Probs: 0.080 (0.137)


In [10]:
testprob

array([[0.08751433, 0.0193342 , 0.01164665],
       [0.32529954, 0.0193342 , 0.01029261],
       [0.01494564, 0.00528675, 0.00404546],
       ...,
       [0.02295547, 0.01525256, 0.00621937],
       [0.10473921, 0.02001447, 0.01029261],
       [0.07783935, 0.02881188, 0.01029261]])

In [11]:
nssi=testprob[:, 1]
nssip=np.where(nssi>0.160344, 1, 0)

In [12]:
def eval(y_hold, ypred, yprob):
    f1= f1_score(y_hold, ypred)
    print(f'F1= {f1:f}')
    sens= recall(y_hold, ypred)
    print(f'Sensitivity= {sens:f}')
    tn, fp, fn, tp= confusion(y_hold, ypred).ravel()
    spec=tn/(tn+fp)
    print(f'Specificity= {spec:f}')
    auc= roc_auc_score(y_hold, yprob)
    print(f'AUROC= {auc:f}')

    return ypred, f1, sens, spec, auc


In [13]:
si=testprob[:, 0]
sip=np.where(si>0.175579, 1, 0)

In [14]:
eval(y_hold_si, sip, si)

F1= 0.456140
Sensitivity= 0.677083
Specificity= 0.804107
AUROC= 0.815643


(array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 

In [15]:
eval(y_hold_nssi, nssip, nssi)

F1= 0.400000
Sensitivity= 0.410714
Specificity= 0.946508
AUROC= 0.806357


(array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [16]:
joblib.dump(chain, 'cclf2802_RF.sav')

['cclf2802_RF.sav']

In [17]:
att=testprob[:, 2]
attp=np.where(att>0.073374, 1, 0)

In [18]:
y_hold_att=y_hold['att']

In [19]:
eval(y_hold_att, attp, att)

F1= 0.325000
Sensitivity= 0.406250
Specificity= 0.949785
AUROC= 0.797391


(array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### Calculating 95% CIs

In [33]:
chain=joblib.load(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\lsac-suicidality\models\cclf2802_RF.sav')

#Defining a function to prepare 100 bootstrapped test sets, and obtain estimates of performance metrics for each resampled version of test data
sre=['si', 'nssi', 'att']
threshs=[0.175579, 0.160344, 0.073374]

def boot(index, thresh):
    #Suppressing warnings for this section. Warning appears because the bootstrapped samples do not have feature names as they are arrays while the model has previously been fitted on pandas dataframes with feature names
    #This is ok because the features are in the same order as the original test set
    warnings.simplefilter(action='ignore', category=UserWarning)
    n_iterations=100
    aucs=list()
    f1s=list()
    sens=list()
    specs=list()
    ppvs=list()
    for i in range(n_iterations):
        Xi_test, ytest=resample(Xi_hold, y_hold, stratify=y_hold, random_state=i)
        probs=chain.predict_proba(Xi_test)
        probs=probs[:, index]
        ytest=ytest.iloc[:, index]
        auc=roc_auc_score(ytest, probs)
        aucs.append(auc)
        pred=np.where(probs>= thresh, 1, 0)
        f1=f1_score(ytest, pred)
        f1s.append(f1)
        sen=recall(ytest, pred)
        sens.append(sen)
        ppv=precision_score(ytest, pred)
        ppvs.append(ppv)
        tn, fp, fn, tp=confusion(ytest, pred).ravel()
        spec=tn/(tn+fp)
        specs.append(spec)
        
    metrics=["aucs", "f1s", "sens", "specs", "ppvs"]
    metricsdf=pd.DataFrame(zip(aucs, f1s, sens, specs, ppvs), columns=metrics)
    return metricsdf

def runevals():
    for idx, (s, t) in enumerate(zip(sre, threshs)):
        print(f'95% CIs for {s} with threshold at {t}')
        metricsdf=boot(idx, t)
        print(metricsdf.quantile([.025, .975]).round(3))
        print('\n')

In [34]:
runevals()

95% CIs for si with threshold at 0.175579
        aucs    f1s   sens  specs   ppvs
0.025  0.776  0.410  0.599  0.782  0.303
0.975  0.863  0.515  0.771  0.832  0.392


95% CIs for nssi with threshold at 0.160344
        aucs    f1s   sens  specs   ppvs
0.025  0.750  0.307  0.304  0.934  0.313
0.975  0.853  0.512  0.546  0.962  0.524


95% CIs for att with threshold at 0.073374
        aucs    f1s   sens  specs   ppvs
0.025  0.737  0.204  0.250  0.937  0.174
0.975  0.863  0.438  0.531  0.967  0.370


